In [1]:
import urllib
from bs4 import BeautifulSoup
import re
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import os
import json
import numpy as np

In [2]:
np.random.randint(1,10,1)[0]

5

In [3]:
#Click on more button
xpath_more="//div[contains(@class,'yt-uix-menu')]"
more_button=driver.find_elements_by_xpath(xpath_more)
more_button[0].text


NameError: name 'driver' is not defined

In [4]:
xpath_transcript="//span[contains(@class,'yt-ui-menu-item-label')]"
transcript_button=driver.find_elements_by_xpath(xpath_transcript)
transcript_button[-3].text

NameError: name 'driver' is not defined

In [5]:
xpath_language="//div[contains(@class,'yt-uix-menu-trigger')]"
lang_button=driver.find_elements_by_xpath(xpath_language)
lang_button[0].text

NameError: name 'driver' is not defined

In [6]:
xpath_english="//span[contains(@class,'yt-uix-button-content transcript-lang')]"
english_button=driver.find_elements_by_xpath(xpath_english)
english_button[1].text

NameError: name 'driver' is not defined

In [7]:
xpath_text="//div[contains(@class,'caption-line-text')]"
text=driver.find_elements_by_xpath(xpath_text)

NameError: name 'driver' is not defined

In [8]:
text[1].text

NameError: name 'text' is not defined

In [9]:
def youtube_transcript_scraper(url):
    """This function scrapes a single transcript from a youtube
    video, along with associated metadata and returns a transcript"""
    driver.get(url)
    time.sleep(np.random.randint(1,8,1)[0])
    
    #Click on more button
    xpath_more="//div[contains(@class,'yt-uix-menu')]"
    more_button=driver.find_elements_by_xpath(xpath_more)
    more_button[0].click()
    time.sleep(1)
    # Click on transcript button
    xpath_transcript="//span[contains(@class,'yt-ui-menu-item-label')]"
    transcript_button=driver.find_elements_by_xpath(xpath_transcript)
    transcript_button[-3].click()
    time.sleep(1)
    # Click on language button and select
    xpath_language="//div[contains(@class,'yt-uix-menu-trigger')]"
    lang_button=driver.find_elements_by_xpath(xpath_language)
    try: 
        lang_button[0].click()
        time.sleep(1)
        # Click the language sub category
        xpath_english="//span[contains(@class,'yt-uix-button-content transcript-lang')]"
        english_button=driver.find_elements_by_xpath(xpath_english)
        english_button[1].click()
        time.sleep(1)
        # Get the text from the transcript
        xpath_text="//div[contains(@class,'caption-line-text')]"
        text=driver.find_elements_by_xpath(xpath_text)
        # Put transcript into a list
        transcript=[]
        print (text[1].text)
        for line in text:
            transcript += line.text.split()
    except:
        transcript=[]
        print('error with:',url)
    

    
    # scrape the title the video
    xpath_title="//span[contains(@class,'watch-title')]"
    video_title=driver.find_elements_by_xpath(xpath_title)
    video_title=video_title[0].text
    
    # Scrape description
    xpath_description="//p[contains(@id,'eow-description')]"
    description=driver.find_elements_by_xpath(xpath_description)
    description=description[0].text
    
    # Scrape number of views
    xpath_views="//div[contains(@class,'watch-view-count')]"
    views=driver.find_elements_by_xpath(xpath_views)
    views=int(''.join(re.findall('[0-9]+',views[0].text)))
    
    # Create a dictionary with the content
    transcript_dict = dict()
    transcript_dict = {
                        'title': video_title,
                        'views': views,
                        'description': description,
                        'transcript': transcript
                        }
    return transcript_dict,video_title

In [10]:
def video_looper(playlist, store_dir):
    for link in playlist:
        transcript_dict,video_title = youtube_transcript_scraper(link)
        filename = ''.join(re.findall('[a-z0-9_]+',video_title.replace(' ','_').lower())) + '.json'
        file_path = store_dir + filename
        with open(file_path, 'w') as fp:
            json.dump(transcript_dict, fp)
    

In [11]:
def playlist_links(playlist_url):
    driver.get(playlist_url)
    #time.sleep(25)
    
    # Title of playlist
    xpath_playlist_title="//h1[contains(@class,'pl-header-title')]"
    playlist_title=driver.find_elements_by_xpath(xpath_playlist_title)
    playlist_title = playlist_title[0].text.replace(' ','_').lower()
    playlist_title = ''.join(re.findall('[a-z0-9_]+',playlist_title))
    
    # Get all links from playlist up to 100
    xpath_links="//td[contains(@class,'pl-video-thumbnail')]//a"
    links=driver.find_elements_by_xpath(xpath_links)
    links_list = [link.get_attribute('href') for link in links]
    
    # create directory for storing
    base_dir = 'youtube/'
    # Make directory if it does not exist in the text folder
    store_dir = base_dir+playlist_title+'/'
    if not os.path.exists(store_dir):
        os.makedirs(store_dir)
    
    # Run video looper to loop through the list of video links
    video_looper(links_list, store_dir)

In [12]:
def multiple_playlist(list_of_playlists):
    for playlist_url in list_of_playlists:
        playlist_links(playlist_url) 

In [13]:
list_of_playlists_first = ['https://www.youtube.com/playlist?list=PLD63A284B7615313A',
                            'https://www.youtube.com/playlist?list=PLA89DCFA6ADACE599',
                          'https://www.youtube.com/playlist?list=PLnvKubj2-I2LhIibS8TOGC42xsD3-liux',
                          'https://www.youtube.com/playlist?list=PL2SOU6wwxB0uwwH80KTQ6ht66KWxbzTIo',
                          'https://www.youtube.com/playlist?list=PLA89DCFA6ADACE599',
                          "https://www.youtube.com/playlist?list=PL3FW7Lu3i5Jsnh1rnUwq_TcylNr7EkRe6",
                           'https://www.youtube.com/playlist?list=PLE7DDD91010BC51F8',
                           "https://www.youtube.com/playlist?list=PL16j5WbGpaM0_Tj8CRmurZ8Kk1gEBc7fg",
                           'https://www.youtube.com/playlist?list=PLrAXtmErZgOeiKm4sgNOknGvNjby9efdf',
                           'https://www.youtube.com/playlist?list=PLkkuNyzb8LmxFutYuPA7B4oiMn6cjD6Rs',
                           'https://www.youtube.com/playlist?list=PLA9E0359014169D37',
                           'https://www.youtube.com/playlist?list=PLUl4u3cNGP61Oq3tWYp6V_F-5jb5L2iHb',
                           'https://www.youtube.com/playlist?list=PLoRl3Ht4JOcdU872GhiYWf6jwrk_SNhz9'
                           
                          ]

In [46]:
list_of_playlists = []
                    


 deep learning
 intro to deep
"https://www.youtube.com/playlist?list=PLZHQObOWTQDPD3MizzM2xVFitgF8hE_ab" essence of line


Not working, no transcript:
    'https://www.youtube.com/playlist?list=PL2SOU6wwxB0v1kQTpqpuu5kEJo2i-iUyf'
    'https://www.youtube.com/playlist?list=PLCJlDcMjVoEdtem5GaohTC1o9HTTFtK7_'
    'https://www.youtube.com/playlist?list=PL1328115D3D8A2566' Khan arabic

In [47]:
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)

In [48]:
multiple_playlist(list_of_playlists)

introduce you to neural networks they're
video on neural networks so this is a
computer set up to run some code so the
our computer in this video we're going
introduce you to something called a cost
to look at finding the slope of our cost
error with: https://www.youtube.com/watch?v=EnGmg-kvpYs&index=7&list=PLxt59R_fWVzT9bDxA76AHm3ig0Gg9S3So
going to look at something called linear
to derive the cost function for linear
going to find the best parameters W and
solve the original problem we started


IndexError: list index out of range

In [ ]:
driver.close()